In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pprint
from pprint import pprint

In [2]:
df = pd.read_csv("C:\\Users\\rohan\\Downloads\\creditcard.csv\\creditcard.csv")
df = df.rename(columns={'Class': 'Fraud'})

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Fraud
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df['Fraud'] = df['Fraud'].astype(int)

X = df.drop(['Fraud'], axis = 1)
Y = df["Fraud"]

xData = X.values
yData = Y.values

In [5]:
xTrain, xTest, yTrain, yTest = train_test_split(
        xData, yData, test_size = 0.2, random_state = 42)

pipe = Pipeline([('standardScaler', StandardScaler()), ('quantiletransformer', QuantileTransformer()), ('xgb_model', xgb.XGBRegressor())])

In [6]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'standardScaler', 'quantiletransformer', 'xgb_model', 'standardScaler__copy', 'standardScaler__with_mean', 'standardScaler__with_std', 'quantiletransformer__copy', 'quantiletransformer__ignore_implicit_zeros', 'quantiletransformer__n_quantiles', 'quantiletransformer__output_distribution', 'quantiletransformer__random_state', 'quantiletransformer__subsample', 'xgb_model__objective', 'xgb_model__base_score', 'xgb_model__booster', 'xgb_model__colsample_bylevel', 'xgb_model__colsample_bynode', 'xgb_model__colsample_bytree', 'xgb_model__enable_categorical', 'xgb_model__gamma', 'xgb_model__gpu_id', 'xgb_model__importance_type', 'xgb_model__interaction_constraints', 'xgb_model__learning_rate', 'xgb_model__max_delta_step', 'xgb_model__max_depth', 'xgb_model__min_child_weight', 'xgb_model__missing', 'xgb_model__monotone_constraints', 'xgb_model__n_estimators', 'xgb_model__n_jobs', 'xgb_model__num_parallel_tree', 'xgb_model__predictor', 'xgb_model__random

In [7]:
from sklearn.model_selection import GridSearchCV

param_grid_pspp = [{
    'xgb_model__objective': ['binary:logistic', 'binary:hinge'],
    'xgb_model__booster': ['gbtree', 'dart'],
    'xgb_model__colsample_bytree': [0.3],# 0.5, 0.7],
    #'xgb_model__gamma': [0, 10, 100, 1000],
    'xgb_model__learning_rate': [0.1],# 0.2, 0.3],
    'xgb_model__max_depth': [5, 6, 7],
    'xgb_model__n_estimators': [8, 10, 12]
    #'xgb_model__reg_alpha': [0 ,5, 10, 15],
    #'xgb_model__reg_lambda': [0, 5, 10, 15]
}]

grid_search_pspp = GridSearchCV(pipe, param_grid_pspp, cv=5,
                                scoring= 'recall', verbose=2, n_jobs=-1)
grid_search_pspp.fit(xTrain, yTrain)


Fitting 5 folds for each of 36 candidates, totalling 180 fits


C:\Users\rohan\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [       nan 0.80552632        nan 0.79346749        nan 0.76513158
        nan 0.80479102        nan 0.78202786        nan 0.77760062
        nan 0.81258514        nan 0.7899226         nan 0.77777864
        nan 0.79052632        nan 0.77013158        nan 0.76233746
        nan 0.7980418         nan 0.77071981        nan 0.76807276
        nan 0.80096749        nan 0.76808824        nan 0.76514706]
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardScaler', StandardScaler()),
                                       ('quantiletransformer',
                                        QuantileTransformer()),
                                       ('xgb_model',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     enable_categorical=False,
                                                     gamma=None, gpu_id=None,
                                                     importance_type=None,
                                                     interaction_constraints=None,
                                           

In [11]:
pprint(grid_search_pspp.best_estimator_)

Pipeline(steps=[('standardScaler', StandardScaler()),
                ('quantiletransformer', QuantileTransformer()),
                ('xgb_model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.3, enable_categorical=False,
                              gamma=0, gpu_id=-1, importance_type=None,
                              interaction_constraints='', learning_rate=0.1,
                              max_delta_step=0, max_depth=7, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=8, n_jobs=20, num_parallel_tree=1,
                              objective='binary:hinge', predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=None, subsample=1,
                              tree_method='exact

In [12]:
pprint(grid_search_pspp.cv_results_)

{'mean_fit_time': array([21.03659239, 22.65926762, 29.26532531, 33.91242285, 48.55281668,
       52.25514712, 36.43403978, 42.38896689, 42.06590977, 45.44814377,
       50.73091149, 59.89149408, 47.86006565, 53.0806654 , 50.65715575,
       56.69640517, 56.52972412, 70.71507387, 36.37240624, 42.40468192,
       57.60131078, 58.10762196, 67.10148849, 66.25685163, 56.0440938 ,
       56.80706639, 60.50241933, 61.74115024, 69.70695004, 71.51275816,
       57.72516446, 61.8824059 , 67.17366347, 64.17221484, 57.55198793,
       49.95115647]),
 'mean_score_time': array([2.64330153, 1.76032686, 4.44676261, 1.99385829, 2.96903715,
       2.67498827, 1.61726041, 1.97274818, 1.51480236, 2.21158957,
       3.17629356, 2.29648948, 1.30930243, 1.419557  , 2.30339098,
       1.44483762, 1.79181762, 2.31988287, 4.34414387, 3.15176764,
       4.36681557, 5.04170518, 5.21592522, 5.48976407, 2.99022031,
       4.50477786, 3.75927296, 5.34031763, 3.72966728, 5.80294633,
       3.80834541, 4.98652368, 3.2